<h2> Import Requirements </h2>

In [ ]:
!pip install datapackage

In [2]:
#Please ensure datapackage is installed
import datapackage
import pandas as pd
import numpy as np
import scipy as sp
import requests
from urllib.request import urlopen

<h2> Obtain Supplementary Datasets </h2>

Before proceeding to the actual preprocessing of S&P500 indicators that would possibly serve as explanatory variables, we must first download a set of supplementary datasets - list of S&P500 companies and macroeconomic indicators to be specific - that would complement micro firm-level data we obtain later in this Notebook

In [3]:
#This is the S&P500 identifier data we require (symbols, names and industry sectors)
data_url = 'https://datahub.io/core/s-and-p-500-companies-financials/datapackage.json'
package = datapackage.Package(data_url)
identifydata = pd.read_csv(package.resources[1].descriptor['path'])
print(identifydata.head(n=5))

  Symbol                 Name                  Sector
0    MMM           3M Company             Industrials
1    AOS      A.O. Smith Corp             Industrials
2    ABT  Abbott Laboratories             Health Care
3   ABBV          AbbVie Inc.             Health Care
4    ACN        Accenture plc  Information Technology


In [4]:
#We must now read in major macroeconomic indicators to supplement our future models
#Out of these, stockconfurl is monthly data and thus must be separated from the yearly reporting of the remainder
stockconfurl = 'https://www.quandl.com/api/v3/datasets/YALE/US_CONF_INDEX_VAL_INDIV.csv?api_key=UxqxiUpQzYRHVDLGsXsz'
gdpurl = 'https://www.quandl.com/api/v3/datasets/WWDI/USA_NY_GDP_MKTP_CD.csv?api_key=UxqxiUpQzYRHVDLGsXsz'
gniurl = 'https://www.quandl.com/api/v3/datasets/WWDI/USA_NY_GNP_MKTP_CD.csv?api_key=UxqxiUpQzYRHVDLGsXsz'
marketcapurl = 'https://www.quandl.com/api/v3/datasets/WWDI/USA_CM_MKT_LCAP_CD.csv?api_key=UxqxiUpQzYRHVDLGsXsz'
confdata = pd.read_csv(stockconfurl, error_bad_lines=False)
confdata.Date = confdata.Date.apply(lambda s: np.int(s.split('-')[0] + s.split('-')[1]))
econurl = [gdpurl, gniurl, marketcapurl]
annualecondata = pd.DataFrame()
for url in econurl:
    data = pd.read_csv(url, error_bad_lines=False)
    data.Date = data.Date.apply(lambda s: np.int(s.split('-')[0] + s.split('-')[1]))
    if annualecondata.empty:
        annualecondata = data.copy()
    else:
        annualecondata = annualecondata.merge(data, on='Date', how='outer')

print(annualecondata.head(n=10))
print()
print(confdata.head(n=5))

     Date       Value_x       Value_y         Value
0  201712  1.939060e+13  1.960760e+13  3.212070e+13
1  201612  1.862448e+13  1.896871e+13  2.735220e+13
2  201512  1.812071e+13  1.858114e+13  2.506754e+13
3  201412  1.742761e+13  1.789207e+13  2.633059e+13
4  201312  1.669152e+13  1.707372e+13  2.403485e+13
5  201212  1.615526e+13  1.659608e+13  1.866833e+13
6  201112  1.551793e+13  1.580290e+13  1.564071e+13
7  201012  1.496437e+13  1.512113e+13  1.728345e+13
8  200912  1.441874e+13  1.449446e+13  1.507729e+13
9  200812  1.471858e+13  1.479119e+13  1.159028e+13

     Date  Index Value  Standard Error
0  201904        49.50            3.52
1  201903        47.83            3.47
2  201902        46.63            3.74
3  201901        47.93            3.84
4  201812        54.48            4.30


In [5]:
identifydata.to_csv('sp500constituents.csv', index=False)
print(identifydata.info())
print()
confdata.to_csv('stockconfidencedata.csv', index=False)
print(confdata.info())
print()
annualecondata.to_csv('annualecondata.csv', index=False)
print(annualecondata.info())
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 3 columns):
Symbol    505 non-null object
Name      505 non-null object
Sector    505 non-null object
dtypes: object(3)
memory usage: 11.9+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 3 columns):
Date              221 non-null int64
Index Value       221 non-null float64
Standard Error    221 non-null float64
dtypes: float64(2), int64(1)
memory usage: 5.3 KB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 57
Data columns (total 4 columns):
Date       58 non-null int64
Value_x    58 non-null float64
Value_y    58 non-null float64
Value      43 non-null float64
dtypes: float64(3), int64(1)
memory usage: 2.3 KB
None



<h2> Define Primary Data's Collection API Functions </h2>

The idea is to first separately retrieve income statement, balance sheet, key metrics and historical stock price data for each S&P500 company. The dividends paid per share in any time duration needs to actually be engineered as Dividend Yield (from key metrics data) * Share Price (from historical stock data), which is done in a later stage. In the second step, an "allinf" function appends all above data for each stock, and then  "sp500inf" code joins 500 such datasets to create our final database for S&P500 firms. 

<b> STEP 1: Create Company-Specific Functions for Different Sets of Financial Data </b>

Since there is a possibility that the API service returns no values for certain stocks, a try-except model is used where an empty dataframe is returned if a function is unable to retrieve appropriate data from the API. The if-else conditions for handling these empty dataframes are built into the "allinf" function

In [6]:
def getincdata (iden):
    incstat = 'https://financialmodelingprep.com/api/financials/income-statement/' + iden + '?datatype=csv'
    try:
        incdata = pd.read_csv(incstat, header=1, error_bad_lines=False)
        incdata = incdata.T
        incdata.columns = incdata.iloc[0,:].values
        indexval = incdata.index.copy()
        incdata.reset_index(drop=True, inplace=True)
        incdata.drop([0,6], axis=0, inplace=True)
        incdata['date'] = indexval[1:len(indexval)-1]
        #We need to adjust columns since there are two columns named 'Basic', 'Diluted' etc., which can hurt further analysis
        incdata = incdata.loc[:,(~incdata.columns.duplicated())]
        incdata.rename(columns={'Basic':'Basic_EPS', 'Diluted':'Diluted_EPS'}, inplace=True)
        return incdata
    except:
        print('Note: ' + iden + ' has an empty csv for income data. Skipping.')
        return pd.DataFrame()

In [7]:
def getbaldata (iden):
    balstat = 'https://financialmodelingprep.com/api/financials/balance-sheet-statement/' + iden + '?datatype=csv'
    try:
        baldata = pd.read_csv(balstat, header=1, error_bad_lines=False)
        baldata = baldata.T
        baldata.columns = baldata.iloc[0,:].values
        indexval = baldata.index.copy()
        baldata.reset_index(drop=True, inplace=True)
        baldata.drop(0, axis=0, inplace=True)
        baldata = baldata.loc[:,(~baldata.columns.duplicated())]
        baldata['date'] = indexval[1:]
        return baldata
    except:
        print('Note: ' + iden + ' has an empty csv for balance sheet data. Skipping.')
        return pd.DataFrame()

In [8]:
def getkeymetrics (iden):
    try:
        metrurl = 'https://financialmodelingprep.com/api/v3/company-key-metrics/' + iden
        metrjson = requests.get(metrurl).json()
        metrdata = pd.io.json.json_normalize(metrjson['metrics'])
        cols = metrdata.columns.values.tolist()
        cols.insert(0, 'date')
        cols = cols[0:len(cols)-1]
        metrdata = metrdata[cols]
        metrdata['date'] = metrdata.date.apply(lambda s: s[:len(s)-3])
        metrdata = metrdata.loc[:,(~metrdata.columns.duplicated())]
        return metrdata
    except:
        print('Note: ' + iden + ' does not have valid key metrics data. Skipping.')
        return pd.DataFrame()

In [9]:
def histstockprc (iden):
    try:
        stockurl = 'https://financialmodelingprep.com/api/v3/historical-price-full/' + iden + '?serietype=line'
        stockjson = requests.get(stockurl).json()
        stockdata = pd.io.json.json_normalize(stockjson['historical'])
        return stockdata
    except:
        print('Note: ' + iden + ' does not have valid stock data. Skipping')
        return pd.DataFrame()

<b> STEP 2: Filter Historic Stock Price Data to Yearly Values </b>

Since all other available data is on a yearly basis, we need to reduce stock price information to yearly info, and add few new features, like price change (3mths), price change (6 mths) and price change (12 mths), to accomodate micro trends. Even though this is feature engineering, it needs to be done at this stage to ensure that the actual dataset can be compiled before any other cleaning is performed

In [10]:
def yearshift(base, amt):
    mth = base % 100
    year = (base - mth)/100
    mth = mth + amt
    if mth > 12:
        year = year + (mth - (mth % 12))/12
        mth = mth % 12
    return int(year * 100 + mth)

In [11]:
def stocksimply (iden):
    try:
        stockdata = histstockprc(iden)
        stockdata['time_stamp'] = stockdata.date.apply(lambda s: np.int(s.split("-")[0] + s.split("-")[1]))
        mth6 = stockdata.copy()
        mth6.time_stamp = mth6.time_stamp.apply(lambda s: yearshift(s, 6))
        mth12 = stockdata.copy()
        mth12.time_stamp = mth12.time_stamp.apply(lambda s: yearshift(s, 12))
        stockdata = stockdata.merge(mth6[["close", "time_stamp"]], on="time_stamp", how="left")
        stockdata = stockdata.merge(mth12[["close", "time_stamp"]], on="time_stamp", how="left")
        stockdata = stockdata[["time_stamp", "date", "close_x", "close_y", "close"]]
        stockdata.rename(columns={"close":"mth12stockratio", "close_y":"mth6stockratio", "close_x":"stockprc"}, inplace=True)
        stockdata.mth6stockratio = stockdata.mth6stockratio.bfill()
        stockdata.mth12stockratio = stockdata.mth12stockratio.bfill()
        stockdata.mth6stockratio = stockdata.stockprc / stockdata.mth6stockratio
        stockdata.mth12stockratio = stockdata.stockprc / stockdata.mth12stockratio
        return stockdata
    except:
        print('Note: ' + iden + ' does not have valid stock data. Skipping')
        return pd.DataFrame()

In [12]:
def selectstock (inc, bal, metr, stock, iden):
    try:
        dateval = np.nan
        if inc.empty == False:
            dateval = inc.date.values
        elif bal.empty == False:
            dateval = bal.date.values
        elif metr.empty == False:
            dateval = metr.date.values
        if isinstance(dateval, np.ndarray):
            stkkp = pd.Series(dateval).apply(lambda s: np.int(s.split("-")[0] + s.split("-")[1])).values
            stock = stock[(stock.time_stamp.isin(stkkp)) & (stock.time_stamp.diff().shift(-1).fillna(1) != 0)]
        else:
            stock = stock[stock.time_stamp.diff().shift(-1).fillna(1) != 0]
        stock.drop("time_stamp", axis=1, inplace=True)
        stock.date = stock.date.apply(lambda s: s[:len(s)-3])
        return stock
    except:
        print('Note: ' + iden + ' does not have valid stock data. Skipping')
        return pd.DataFrame()

<b> STEP 3: Define all Variables to be Retreived from API Calls </b>

It is possible that the Financial Modelling Prep API that we use does not return all columns for all stocks. For example, a baldata call on 'MMM' returns 55 columns while one on 'AOS' omits 10 and gives 45 columns instead. Since all companies need to have the same variables to be appended into our initial S&P500 dataset, all non-returned variables are currently set to np.nan, and then handled later on during data cleaning

In [13]:
allvar = ['date', 'Revenue', 'Cost of revenue', 'Gross profit', 'Operating expenses', 
          'Research and development', 'Sales, General and administrative',
          'Total operating expenses', 'Operating income', 'Interest Expense',
          'Other income (expense)', 'Income before taxes','Provision for income taxes',
          'Net income from continuing operations', 'Other', 'Net income',
          'Net income available to common shareholders','Earnings per share', 'Basic_EPS', 
          'Diluted_EPS','Weighted average shares outstanding','EBITDA', 'Assets', 
          'Current assets', 'Cash','Cash and cash equivalents', 'Short-term investments',
          'Total cash', 'Receivables', 'Inventories','Prepaid expenses',
          'Other current assets', 'Total current assets','Non-current assets', 
          'Property, plant and equipment','Gross property, plant and equipment', 
          'Accumulated Depreciation','Net property, plant and equipment',
          'Equity and other investments', 'Goodwill', 'Intangible assets', 
          'Prepaid pension benefit','Other long-term assets', 'Total non-current assets',
          'Total assets', "Liabilities and stockholders' equity",'Liabilities', 
          'Current liabilities', 'Short-term debt','Accounts payable', 'Taxes payable',
          'Accrued liabilities', 'Other current liabilities', 'Total current liabilities',
          'Non-current liabilities', 'Long-term debt','Capital leases',
          'Deferred taxes liabilities','Pensions and other benefits', 'Minority interest',
          'Other long-term liabilities', 'Total non-current liabilities','Total liabilities', 
          "Stockholders' equity", 'Common stock','Additional paid-in capital', 
          'Retained earnings','Treasury stock', 'Accumulated other comprehensive income',
          "Total stockholders' equity","Total liabilities and stockholders' equity", 
          'Average Inventory','Average Payables', 'Average Receivables', 'Book Value per Share',
          'Capex per Share', 'Capex to Depreciation','Capex to Operating Cash Flow', 
          'Capex to Revenue','Cash per Share', 'Current ratio', 'Days Payables Outstanding',
          'Days Sales Outstanding', 'Days of Inventory on Hand','Debt to Assets', 
          'Debt to Equity', 'Dividend Yield','EV to Free cash flow', 
          'EV to Operating cash flow', 'EV to Sales','Earnings Yield', 'Enterprise Value',
          'Enterprise Value over EBITDA', 'Free Cash Flow Yield','Free Cash Flow per Share', 
          'Graham Net-Net', 'Graham Number','Income Quality', 'Intangibles to Total Assets',
          'Interest Coverage', 'Interest Debt per Share','Inventory Turnover', 
          'Invested Capital', 'Market Cap','Net Current Asset Value', 'Net Debt to EBITDA',
          'Net Income per Share', 'Operating Cash Flow per Share','PB ratio', 'PE ratio', 
          'PFCF ratio', 'POCF ratio', 'PTB ratio','Payables Turnover', 'Payout Ratio', 
          'Price to Sales Ratio','R&D to Revenue', 'ROE', 'ROIC', 'Receivables Turnover',
          'Return on Tangible Assets', 'Revenue per Share','SG&A to Revenue', 
          'Shareholders Equity per Share','Stock-based compensation to Revenue', 
          'Tangible Asset Value','Tangible Book Value per Share', 'Working Capital', 
          'stockprc','mth6stockratio', 'mth12stockratio']
len(allvar)

131

<b> STEP 4: Aggregate Collected Data </b>

In [14]:
def allinf (iden):
    incdata = getincdata(iden)
    baldata = getbaldata(iden)
    metrdata = getkeymetrics(iden)
    stockdata = stocksimply(iden)
    #Error checking for all the data retreival functions
    if (isinstance(incdata, pd.DataFrame) & isinstance(baldata, pd.DataFrame) & isinstance(metrdata, pd.DataFrame) & isinstance(stockdata, pd.DataFrame)) == False:
        print("Error occured in reading data")
        return pd.DataFrame()
    stockdata = selectstock(incdata, baldata, metrdata, stockdata, iden)
    if (incdata.empty) | (baldata.empty):
        incdata = incdata.append(baldata, ignore_index=True)
    else:
        incdata = incdata.merge(baldata, on="date", how="inner")
    if (incdata.empty) | (metrdata.empty):
        incdata = incdata.append(metrdata, ignore_index=True)
    else:
        incdata = incdata.merge(metrdata, on="date", how = "inner")
    if (incdata.empty) | (stockdata.empty):
        incdata = incdata.append(stockdata, ignore_index=True)
    else:
        incdata = incdata.merge(stockdata, on="date", how="inner")
    avcols = incdata.columns.values
    misscols = list(set(allvar)-set(avcols))
    for i in misscols:
        incdata[i] = np.nan
    incdata = incdata[allvar]
    return incdata

In [15]:
#sp500inf: get all required S&P500 data into one single table
member = pd.read_csv("constituents_csv.csv")
symbols = member['Symbol'].values
alldata = allinf(symbols[0])
alldata['firm'] = symbols[0]
for i in range(1, len(symbols)):
    temp = allinf(symbols[i])
    temp['firm'] = symbols[i]
    alldata = alldata.append(temp, ignore_index=True)
    if (i % 20 == 0):
        print(i)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Note: ATVI has an empty csv for income data. Skipping.
Note: ATVI has an empty csv for balance sheet data. Skipping.
Note: ADBE has an empty csv for income data. Skipping.
Note: AMD has an empty csv for income data. Skipping.
Note: AET does not have valid key metrics data. Skipping.
Note: ALXN has an empty csv for income data. Skipping.
Note: ALXN has an empty csv for balance sheet data. Skipping.
20
Note: AGN has an empty csv for income data. Skipping.
Note: AGN has an empty csv for balance sheet data. Skipping.
Note: GOOG does not have valid key metrics data. Skipping.
Note: AMZN has an empty csv for income data. Skipping.
Note: AEE has an empty csv for balance sheet data. Skipping.
Note: AMP has an empty csv for balance sheet data. Skipping.
Note: AME has an empty csv for balance sheet data. Skipping.
40
Note: ANDV does not have valid key metrics data. Skipping.
Note: ANTM has an empty csv for income data. Skipping.
Note: ANTM has an empty csv for balance sheet data. Skipping.
60
No

Note: CLX has an empty csv for balance sheet data. Skipping.
440
Note: TWX does not have valid key metrics data. Skipping.
Note: UDR has an empty csv for balance sheet data. Skipping.
Note: UA has an empty csv for income data. Skipping.
Note: UA has an empty csv for balance sheet data. Skipping.
Note: UA does not have valid key metrics data. Skipping.
460
Note: VTR has an empty csv for income data. Skipping.
Note: VTR has an empty csv for balance sheet data. Skipping.
480
Note: HCN has an empty csv for income data. Skipping.
Note: HCN does not have valid key metrics data. Skipping.
Note: HCN does not have valid stock data. Skipping
Note: HCN does not have valid stock data. Skipping
Note: HCN does not have valid stock data. Skipping
Note: WLTW has an empty csv for income data. Skipping.
Note: WLTW has an empty csv for balance sheet data. Skipping.
Note: WYN does not have valid key metrics data. Skipping.
Note: XL does not have valid key metrics data. Skipping.
500


In [16]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2566 entries, 0 to 2565
Columns: 132 entries, date to firm
dtypes: float64(3), object(129)
memory usage: 2.6+ MB


In [17]:
alldata.to_csv("sp500alldata.csv", index=False, header=True)